In [ ]:
"""
Импортируем необходимые библиотеки и функции для работы с LSTM и предобученным трансформером:
- os, datetime: работа с файловой системой и временем
- torch, nn, optim: PyTorch и нейронные сети
- Tokenizer: токенизация текста
- get_all_dataloaders: загрузка датасетов для LSTM
- LSTMModel: модель LSTM
- train_one_epoch: обучение LSTM по эпохам
- lstm_evaluate, lstm_generate: оценка LSTM и генерация примеров
- transformer_evaluate, transformer_generate: генерация и оценка GPT-2
"""
import os
from datetime import datetime
import torch
from torch import nn, optim

from tokenizers import Tokenizer
from src.next_token_dataset import get_all_dataloaders
from src.lstm_model import LSTMModel   
from src.lstm_train import train_one_epoch
from src.lstm_eval import lstm_evaluate, lstm_generate
from src.transformer_pipiline_eval import transformer_evaluate, transformer_generate

In [15]:
"""
Определяем устройство для вычислений: GPU (CUDA), если доступен, иначе CPU.
Выводим, какое устройство используется для обучения и генерации.
"""
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("Используем device:", device)

Используем device: cpu


In [ ]:
"""
Задаём основные гиперпараметры модели и обучения:
- vocab_size: размер словаря для эмбеддингов;
- batch_size: размер батча для загрузчика данных;
- embedding_dim: размерность векторного представления слов;
- hidden_dim: количество скрытых нейронов в LSTM;
- num_layers: число слоёв LSTM;
- learning_rate: скорость обучения оптимизатора;
- dropout: вероятность отключения нейронов для регуляризации;
- epochs: количество эпох обучения.
"""
vocab_size = 20000
batch_size = 256
embedding_dim = 128
hidden_dim = 128
num_layers = 1
learning_rate = 1e-3
dropout = 0.1
epochs = 10

In [ ]:
"""
Создаём загрузчики данных для обучения, валидации и тестирования.

train_loader — для обучения модели,
val_loader — для вычисления метрик на валидационном наборе,
test_loader — для оценки на тестовом наборе после обучения.
"""
train_path = 'data/train.txt'
val_path = 'data/val.txt'
test_path = 'data/test.txt'

train_loader, val_loader, test_loader = get_all_dataloaders(train_path, 
                                                            val_path, 
                                                            test_path, 
                                                            batch_size)

In [ ]:
"""
Загружаем BPE-токенизатор из файла и создаём словарь idx2word.
Этот словарь позволяет переводить числовые токены обратно в текстовые токены
"""
tokenizer = Tokenizer.from_file("tokenizer/bpe_tokenizer.json")
idx2word = {i: tokenizer.id_to_token(i) for i in range(tokenizer.get_vocab_size())}

In [ ]:
"""
Создаём экземпляр LSTM модели с заданными гиперпараметрами (размер словаря, размер эмбеддингов, 
размер скрытого состояния, количество слоёв и dropout) и переносим модель на выбранное устройство.
"""
model = LSTMModel(vocab_size, embedding_dim, hidden_dim, num_layers, dropout).to(device)

In [ ]:
"""
Определяем функцию потерь для задачи классификации токенов (CrossEntropyLoss) 
и оптимизатор Adam для обновления весов модели с заданной скоростью обучения.
"""
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
"""
Функция train_model выполняет полный цикл обучения LSTM-модели:
1. Создает уникальную папку для сохранения весов текущего запуска.
2. Для каждой эпохи:
   a) Обучает модель на тренировочном датасете и выводит loss.
   b) Валидирует модель на валидационном датасете, вычисляет loss и ROUGE-1 метрику.
   c) Сохраняет веса модели для текущей эпохи.
   d) Генерирует примеры текстов (три твита) для оценки качества генерации.
3. Использует токенизатор для преобразования текста в токены и обратно.
"""
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs, device, idx2word=None):
    timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    run_folder = os.path.join("models", f"run_{timestamp}")
    os.makedirs(run_folder, exist_ok=True)
    
    for epoch in range(epochs):
        # обучение
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}", flush=True)

        # валидация
        val_loss, rouge_score = lstm_evaluate(model, val_loader, criterion, device, idx2word)
        print(f"Val Loss: {val_loss:.4f}, ROUGE: {rouge_score:.4f}", flush=True)

        # сохраняем веса
        save_path = os.path.join(run_folder, f"lstm_model_epoch{epoch+1}.pt")
        torch.save(model.state_dict(), save_path)

        # генерируем пример твита
        model.eval()
        with torch.no_grad():
            example_texts = [
                "The weather today is amazing",
                "I just watched a movie, it was",
                "Learning machine learning is fun and exciting",
            ]

            for i, text in enumerate(example_texts):
                # Токенизируем
                encoding = tokenizer.encode(text)
                prompt_tokens = torch.tensor([encoding.ids], device=device)

                # Создаем последовательность
                generated = model.generate(prompt_tokens, max_len=4)

                # Декодируем
                decoded_text = tokenizer.decode(generated[0].tolist(), skip_special_tokens=True)

                print(f"Tweet {i+1} generated after epoch {epoch+1}: {decoded_text}\n", flush=True)

In [ ]:
"""
Запуск обучения модели
"""
train_model(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    epochs,
    device,
    idx2word
)

In [ ]:
"""
Загрузка сохраненных весов обученной LSTM-модели:
1. weights_path — путь к файлу с весами модели последней эпохи.
2. load_state_dict загружает веса в модель на указанное устройство (CPU или GPU).
3. model.eval() переводит модель в режим инференса (отключает dropout и другие тренировочные механизмы).
"""
weights_path = "models/run_2025_08_20_06_59_34/lstm_model_epoch10.pt"
model.load_state_dict(torch.load(weights_path, map_location=device))
model.eval()

In [ ]:
"""
Текст для геренации lstm и transformer для сравнения
"""
example_texts = [
    "The weather today is amazing",
    "I just watched a movie, it was",
    "Learning machine learning is fun and exciting",
]    

In [ ]:
"""
Генерация текста на примере нескольких твитов с использованием:
1. lstm_generate — генерация продолжений с обученной LSTM-моделью.
2. transformer_generate — генерация продолжений с предобученной моделью DistilGPT-2.
Используются одинаковые примеры, чтобы визуально сравнить качество генерации.
"""
lstm_generate(example_texts, tokenizer, model, device)
transformer_generate(example_texts)

In [ ]:
"""
Валидация предобученной модели DistilGPT-2 на всём датасете:
1. Загружаются строки из raw_dataset.txt.
2. Для каждой строки модель генерирует продолжение.
3. Вычисляется среднее значение метрики ROUGE-1 для всех сгенерированных текстов.
Это позволяет сравнить качество генерации GPT-2 обученной LSTM-моделью.
"""
with open("data/raw_dataset.txt", "r", encoding="utf-8") as f:
    val_texts = [line.strip() for line in f if line.strip()]

avg_rouge1 = transformer_evaluate(val_texts)

### Сравнение LSTM и предобученного трансформера (DistilGPT2)

#### 1. LSTM: метрики по эпохам

| Эпоха | Train Loss | Val Loss | ROUGE-1 |
|-------|------------|----------|---------|
| 1/10  | 6.7318     | 6.0905   | 0.1250  |
| 2/10  | 6.0621     | 5.9105   | 0.1315  |
| 3/10  | 6.0109     | 5.8629   | 0.1366  |
| 4/10  | 5.9073     | 5.8732   | 0.1410  |
| 5/10  | 5.9176     | 5.9312   | 0.1466  |
| 6/10  | 6.0484     | 5.9877   | 0.1503  |
| 7/10  | 6.0137     | 6.0257   | 0.1497  |
| 8/10  | 6.0844     | 6.0691   | 0.1551  |
| 9/10  | 6.2463     | 6.1460   | 0.1492  |
| 10/10 | 6.2086     | 6.1846   | 0.1544  |

**Примеры генерации LSTM после 10-й эпохи:**

- Prompt: *The weather today is amazing*  
  Generated: *the weather today is amazing that ' s , sad that has the eee had*

- Prompt: *I just watched a movie, it was*  
  Generated: *i just watched a movie , it was great do you have at ' ll to self and*

- Prompt: *Learning machine learning is fun and exciting*  
  Generated: *learning machine learning is fun and exciting will morning , won ' t some on my little*

---

#### 2. Предобученный трансформер DistilGPT2

**Примеры генерации на тех же промптах:**

- Prompt: *The weather today is amazing*  
  Generated: *The weather today is amazing. Weather data from the National Weather Service is available*

- Prompt: *I just watched a movie, it was*  
  Generated: *I just watched a movie, it was a lot of fun," he said. "I*

- Prompt: *Learning machine learning is fun and exciting*  
  Generated: *Learning machine learning is fun and exciting. It›s about teaching a new way*

**Примеры генерации на raw датасете твитов:**

- Prefix: *is upset that he can’t update his Facebook by texting it… and might cry as a resu*  
  Target: *lt  School today also. Blah!*  
  Generated: *ptor, but he’s still not sure how to respond to this.*  

- Prefix: *@Kenichan I dived many times for the ball. Managed to save 50%  Th*  
  Target: *e rest go out of bounds*  
  Generated: *umbs down from the field during the first half. Can’t get too close to the ball.*  

**Среднее значение ROUGE-1 на валидационном датасете:** `0.0476`

---

#### 3. Вывод

- LSTM показывает **лучший ROUGE-1 (≈0.15)** на валидационном наборе, чем предобученный DistilGPT2 (≈0.048) на тех же данных.  
- LSTM генерирует менее «связные» предложения по смыслу, но ближе к специфике тренировочного датасета.  
- Предобученный трансформер генерирует грамматически корректные и более «естественные» тексты, но не адаптирован к конкретной задаче автодополнения твитов.

> Примечания:
> - LSTM обучался на 10 эпохах на конкретном датасете твитов, поэтому его генерации более приближены к стилю тренировочных данных.  
> - Предобученный DistilGPT2 не дообучался на этом датасете, поэтому его генерации естественные, но менее похожи на конкретный набор данных.  
> - ROUGE-1 измеряет совпадение токенов с целевыми текстами; LSTM показывает выше метрику, потому что «учился» именно на этих твитах.  

---